## Problem 51
https://projecteuler.net/problem=51

In [1]:
using Primes, Combinatorics, BenchmarkTools

In [2]:
function undigit(l::Array{Int64, 1})::Int64
    """

    julia> undigit([3, 4, 5])
    345
    
    """
    @assert all(n -> n ∈ 0:9, l) "Only integers in range 0:9 are valid inputs"
    s = 0 # sum
    for (i, n) in enumerate(reverse(l))
        s += n * 10^(i-1)
    end
    return s
end

undigit (generic function with 1 method)

In [3]:
undigit([1, 0, 0, 0])

1000

In [4]:
function replace_digits(n, d, idx)
    """

    julia> replace_digits(9999, 0, [2, 3])
    9009
    
    """
    digits_of_n = reverse(digits(n))
    digits_of_n[idx] .= d
    return undigit(digits_of_n)
end

replace_digits (generic function with 1 method)

In [5]:
replace_digits(111111, 9, [3, 4])

119911

In [6]:
[replace_digits(111111, d, [3, 4]) for d in 0:9]

10-element Vector{Int64}:
 110011
 111111
 112211
 113311
 114411
 115511
 116611
 117711
 118811
 119911

### Solution 1

In [7]:
function prime_digit_replacements(treshold, n=1, info=false)
    n = 1 # start with primes of length n
    while true
        for p in primes(10^(n-1), (10^n)-1) # generate primes of length n
            for index_combination in powerset(1:n) # generate all index combinations in range 1:n
                valid_digits = 1 ∉ index_combination ? (0:9) : (1:9) # do not replace first digit with zero
                new_values = [replace_digits(p, d, index_combination) for d in valid_digits]
                if sum(isprime, new_values) == treshold && p in new_values
                    if info
                        new_primes = [x for x in new_values if isprime(x)]
                        println("Values changed for positions: ", index_combination),
                        println("Prime family: ", new_primes)
                    end
                    return p
                end
            end
        end
        n += 1
    end
end

prime_digit_replacements (generic function with 3 methods)

In [8]:
@assert prime_digit_replacements(7, 1, true) == 56003

Values changed for positions: [3, 4]
Prime family: [56003, 56113, 56333, 56443, 56663, 56773, 56993]


In [9]:
prime_digit_replacements(7)

56003

In [10]:
@btime prime_digit_replacements(8)

  780.944 ms (12780777 allocations: 1.18 GiB)


121313

#### Solution 2

In [11]:
function prime_digit_replacements_2(treshold, n=1)
    for i in Iterators.countfrom(n), p in primes(10^(i-1), (10^i)-1), index_combination in powerset(1:i)
        new_values = [replace_digits(p, d, index_combination) for d in (1 ∉ index_combination ? (0:9) : (1:9))]
        sum(isprime, new_values) == treshold && p in new_values && return p
    end
end

prime_digit_replacements_2 (generic function with 2 methods)

In [12]:
@btime prime_digit_replacements_2(8)

  772.210 ms (12780777 allocations: 1.18 GiB)


121313

In [13]:
ndigits(prime_digit_replacements_2(8))

6

In [14]:
@btime prime_digit_replacements_2(8, 6)

  288.922 ms (3692272 allocations: 385.60 MiB)


121313